In [7]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/out.zip .
!unzip out.zip
!rm out.zip
!mv out.csv data.csv

Mounted at /content/drive
Archive:  out.zip
  inflating: out.csv                 


In [8]:
import pandas as pd

df = pd.read_csv('data.csv')
df.head()

,street,highCard,avgRank,flushPossible,boardPaired,hasFour,lc3flush,lc4flush,lc4straight,lcOvercard,plAgg,opAgg,numAggCS,isLastAgg,spr,amtToCall,numPlayers,relPos,result
0,1,10,5.333333,False,False,False,False,False,False,False,2.000000,0.333333,0,True,11.111111,0.000000,2,0.5,C
1,1,10,5.333333,False,False,False,False,False,False,False,0.333333,1.000000,0,False,11.111111,0.000000,2,1.0,C
2,2,10,5.250000,False,False,False,False,False,False,False,1.000000,0.250000,0,True,11.111111,0.000000,2,0.5,R
3,2,10,5.250000,False,False,False,False,False,False,False,0.250000,1.500000,1,False,7.074074,0.333333,2,1.0,F
4,1,13,9.666667,False,True,False,False,False,False,False,0.500000,2.000000,0,False,15.076923,0.000000,2,0.5,C


In [9]:
def normMinMax(columns):
    for col in columns:
        minVal = df[col].min()
        maxVal = df[col].max()
        df[col] = df[col].apply(lambda x: (x - minVal) / (maxVal - minVal))

def normUnbounded(columns):
    for col in columns:
        df[col] = df[col].apply(lambda x: x / (1 + x))

def boolToInt(columns):
    for col in columns:
        df[col] = df[col].apply(lambda x: 1 if x == True else 0)

def convertLabels(x):
    if x == 'F':
        return 0
    elif x == 'C':
        return 1
    elif x == 'R':
        return 2
    else:
        raise Exception('Uknown label')

In [10]:
cols = ['street', 'highCard', 'avgRank', 'numPlayers', 'relPos']
normMinMax(cols)

cols = ['plAgg', 'opAgg', 'numAggCS', 'spr']
normUnbounded(cols)

cols = ['flushPossible', 'boardPaired', 'hasFour', 'lc3flush', 'lc4flush', 'lc4straight', 'lcOvercard', 'isLastAgg']
boolToInt(cols)

df['result'] = df['result'].apply(convertLabels)

In [11]:
df.head()

,street,highCard,avgRank,flushPossible,boardPaired,hasFour,lc3flush,lc4flush,lc4straight,lcOvercard,plAgg,opAgg,numAggCS,isLastAgg,spr,amtToCall,numPlayers,relPos,result
0,0.0,0.75,0.361111,0,0,0,0,0,0,0,0.666667,0.250000,0.0,1,0.917431,0.000000,0.0,0.4,1
1,0.0,0.75,0.361111,0,0,0,0,0,0,0,0.250000,0.500000,0.0,0,0.917431,0.000000,0.0,1.0,1
2,0.5,0.75,0.354167,0,0,0,0,0,0,0,0.500000,0.200000,0.0,1,0.917431,0.000000,0.0,0.4,2
3,0.5,0.75,0.354167,0,0,0,0,0,0,0,0.200000,0.600000,0.5,0,0.876147,0.333333,0.0,1.0,0
4,0.0,1.00,0.722222,0,1,0,0,0,0,0,0.333333,0.666667,0.0,0,0.937799,0.000000,0.0,0.4,1


In [12]:
len(df)

26199925

In [13]:
df['result'].value_counts()

1    15078225
2     6722445
0     4399255
Name: result, dtype: int64

In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras

def splitData(df, seed=123):
    y = df['result'].to_numpy(dtype=np.uint8, copy=True)
    x = df.drop(['result'], axis=1).to_numpy(dtype=np.float32, copy=True)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=seed)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=1/9, random_state=seed)
    y_train = keras.utils.to_categorical(y_train, 3)
    y_val = keras.utils.to_categorical(y_val, 3)
    y_test = keras.utils.to_categorical(y_test, 3)
    return x_train, x_val, x_test, y_train, y_val, y_test

In [15]:
x_train, x_val, x_test, y_train, y_val, y_test = splitData(df)

In [45]:
def createModel(steps=1, lr=0.001):
    optimizer = keras.optimizers.RMSprop(learning_rate=lr)
    model = keras.Sequential([
        keras.Input(shape=(18,)),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(12, activation='relu'),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(6, activation='relu'),
        keras.layers.Dense(3, activation='softmax')
    ])
    model.compile(
        loss='categorical_crossentropy',
        optimizer='rmsprop',
        metrics=[keras.metrics.Precision(), keras.metrics.Recall()],
        steps_per_execution=steps
    )
    return model

In [28]:
def trainModel(model, x_train, y_train, x_val, y_val, filepath='model.keras', batch_size=256, epochs=10):
    callbacks = [
        keras.callbacks.ModelCheckpoint(
            filepath=filepath,
            save_best_only=True,
            monitor='val_loss'
        )
    ]
    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_val, y_val),
        callbacks=callbacks
    )
    return callbacks, history

In [43]:
def test_model(filename, x_test, y_test, batch_size=2**14):
    model = keras.models.load_model(filename)
    loss, precision, recall = model.evaluate(x_test, y_test, batch_size=batch_size)
    print(f'Loss: {loss:.3f},', f'Precision: {precision:.3f},', f'Recall: {recall:.3f}')

In [29]:
def get_optimal_epoch(history):
    val_loss = history.history['val_loss']
    return val_loss.index(min(val_loss)) + 1

In [20]:
import tensorflow as tf
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
strategy = tf.distribute.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.78.25.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.78.25.50:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with tf.distribute.TPUStrategy(tpu).scope():
    model = createModel(steps=8, lr=0.005)

model.summary()

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_68 (Dense)            (None, 16)                304       
                                                                 
 dense_69 (Dense)            (None, 12)                204       
                                                                 
 dense_70 (Dense)            (None, 8)                 104       
                                                                 
 dense_71 (Dense)            (None, 6)                 54        
                                                                 
 dense_72 (Dense)            (None, 3)                 21        
                                                                 
Total params: 687
Trainable params: 687
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks, history = trainModel(model, x_train, y_train, x_val, y_val, filepath='model-tpu.keras', batch_size=2**14, epochs=100)

Epoch 1/100
1280/1280 [==============================] - 24s 19ms/step - loss: 0.7054 - precision_6: 0.6632 - recall_6: 0.5935 - val_loss: 0.6737 - val_precision_6: 0.6787 - val_recall_6: 0.6221
Epoch 2/100
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6729 - precision_6: 0.6792 - recall_6: 0.6275 - val_loss: 0.6704 - val_precision_6: 0.6807 - val_recall_6: 0.6315
Epoch 3/100
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6703 - precision_6: 0.6806 - recall_6: 0.6322 - val_loss: 0.6687 - val_precision_6: 0.6805 - val_recall_6: 0.6366
Epoch 4/100
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6682 - precision_6: 0.6818 - recall_6: 0.6348 - val_loss: 0.6664 - val_precision_6: 0.6828 - val_recall_6: 0.6365
Epoch 5/100
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6668 - precision_6: 0.6824 - recall_6: 0.6367 - val_loss: 0.6661 - val_precision_6: 0.6828 - val_recall_6: 0.6352
Epoch 6/100
1280/1280 [======

In [ ]:
get_optimal_epoch(history)

84

In [35]:
with strategy.scope():
    model = createModel(steps=8, lr=0.005)

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 3)                 57        
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [37]:
callbacks, history = trainModel(model, x_train, y_train, x_val, y_val, filepath='linear.keras', batch_size=2**14, epochs=30)

Epoch 1/30
1280/1280 [==============================] - 15s 11ms/step - loss: 0.6875 - precision_5: 0.6616 - recall_5: 0.6166 - val_loss: 0.6873 - val_precision_5: 0.6591 - val_recall_5: 0.6164
Epoch 2/30
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6875 - precision_5: 0.6615 - recall_5: 0.6166 - val_loss: 0.6873 - val_precision_5: 0.6600 - val_recall_5: 0.6168
Epoch 3/30
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6875 - precision_5: 0.6616 - recall_5: 0.6166 - val_loss: 0.6874 - val_precision_5: 0.6658 - val_recall_5: 0.6188
Epoch 4/30
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6875 - precision_5: 0.6616 - recall_5: 0.6168 - val_loss: 0.6874 - val_precision_5: 0.6656 - val_recall_5: 0.6190
Epoch 5/30
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6875 - precision_5: 0.6615 - recall_5: 0.6168 - val_loss: 0.6873 - val_precision_5: 0.6617 - val_recall_5: 0.6171
Epoch 6/30
1280/1280 [============

In [42]:
test_model('linear.keras', x_test, y_test)

160/160 [==============================] - 2s 11ms/step - loss: 0.6878 - precision_5: 0.6604 - recall_5: 0.6166
Loss: 0.688, Precision: 0.660, Recall: 0.617


In [46]:
with strategy.scope():
    model = createModel(steps=8, lr=0.005)

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 16)                304       
                                                                 
 dense_14 (Dense)            (None, 12)                204       
                                                                 
 dense_15 (Dense)            (None, 8)                 104       
                                                                 
 dense_16 (Dense)            (None, 6)                 54        
                                                                 
 dense_17 (Dense)            (None, 3)                 21        
                                                                 
Total params: 687
Trainable params: 687
Non-trainable params: 0
_________________________________________________________________


In [47]:
callbacks, history = trainModel(model, x_train, y_train, x_val, y_val, filepath='model-tpu.keras', batch_size=2**14, epochs=300)

Epoch 1/300
1280/1280 [==============================] - 24s 18ms/step - loss: 0.7108 - precision_6: 0.6640 - recall_6: 0.5910 - val_loss: 0.6720 - val_precision_6: 0.6782 - val_recall_6: 0.6306
Epoch 2/300
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6710 - precision_6: 0.6794 - recall_6: 0.6305 - val_loss: 0.6698 - val_precision_6: 0.6803 - val_recall_6: 0.6285
Epoch 3/300
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6679 - precision_6: 0.6814 - recall_6: 0.6325 - val_loss: 0.6657 - val_precision_6: 0.6834 - val_recall_6: 0.6339
Epoch 4/300
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6661 - precision_6: 0.6819 - recall_6: 0.6349 - val_loss: 0.6645 - val_precision_6: 0.6824 - val_recall_6: 0.6381
Epoch 5/300
1280/1280 [==============================] - 11s 9ms/step - loss: 0.6651 - precision_6: 0.6824 - recall_6: 0.6368 - val_loss: 0.6637 - val_precision_6: 0.6829 - val_recall_6: 0.6384
Epoch 6/300
1280/1280 [======

In [48]:
get_optimal_epoch(history)

293

In [50]:
test_model('model-tpu.keras', x_test, y_test)

160/160 [==============================] - 2s 12ms/step - loss: 0.6568 - precision_6: 0.6866 - recall_6: 0.6455
Loss: 0.657, Precision: 0.687, Recall: 0.645


In [51]:
!cp model-tpu.keras /content/drive/MyDrive/model.keras